In [1]:
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.wv.vectors.npy
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.trainables.syn1neg.npy
!pip install fastparquet

--2024-06-18 21:25:10--  https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
Resolving huggingface.co (huggingface.co)... 54.192.18.37, 54.192.18.10, 54.192.18.113, ...
Connecting to huggingface.co (huggingface.co)|54.192.18.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e8/72/e872abef7b2278b5cf6a4f41d1455f5d309cdbd6310b2da0a6e9aecc840217e7/7f920b29a2c3861a475f296d397ff679528b064dbb51debf33568ae5f4f7e088?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27w2v_OA_CR_100d.bin%3B+filename%3D%22w2v_OA_CR_100d.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1719005110&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTAwNTExMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lOC83Mi9lODcyYWJlZjdiMjI3OGI1Y2Y2YTRmNDFkMTQ1NWY1ZDMwOWNkYmQ2MzEwYjJkYTBhNmU5YWVjYzg0MDIxN2U3LzdmOTIwYjI5YTJjMzg2MWE0N

In [2]:
from gensim.models import FastText, Word2Vec, KeyedVectors
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from nltk.stem import WordNetLemmatizer
import string
import nltk
from torch.utils.data import Dataset,DataLoader, random_split
import re
from numpy import random
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
model = Word2Vec.load('w2v_OA_CR_100d.bin')
print(model.wv.get_vector('lymphangioleiomyomatosis'))

[-0.6044092   0.4017609  -0.717026   -0.2701869  -0.23817156 -0.34242344
 -0.15332928 -0.11580862 -0.36377856 -0.10749034  0.4498769   0.7072215
 -0.4689228  -0.48433578  1.050483    0.71655744 -0.6439546  -0.44996452
 -0.11657333  0.09194979  0.35634688  0.4884644   0.28647774 -0.496315
 -0.7018925  -0.25540805  0.14061369 -0.8933765  -0.52663106  0.40501425
  0.17843099  0.21091795 -0.231396   -0.18487869  0.38206643  0.09275728
 -0.4573524   0.7668036   0.17861798  0.31376782 -0.49365893 -0.5377006
  0.27592292 -0.04515044 -0.23638678 -0.5425362  -0.46474802  0.10646814
 -0.17016436 -0.07704978  0.17462458  0.27287003  0.5647319   0.1521702
  0.1517999   0.2716373   0.21111147 -0.11035519  0.19445771  0.14426446
  0.7005133  -0.71069545  0.01735174  0.30701727 -0.54055333 -0.0802884
 -0.1946127  -0.4616384  -0.4518422  -0.00218993 -0.3471012   0.60170263
 -0.47892392 -0.04716599  0.07105272  0.35743713 -0.4405513  -0.53299844
 -0.21108003  0.10141873  0.25599658 -0.22999842  0.21882

In [4]:
test_data = pd.read_parquet('test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')
test_data

,id,query,answer,choices,gold
0,MedNLI0,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1,MedNLI1,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
2,MedNLI2,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
3,MedNLI3,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
4,MedNLI4,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
...,...,...,...,...,...
1417,MedNLI1417,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1
1418,MedNLI1418,\nTASK: Please classify the relationship betwe...,neutral,"[entailment, contradiction, neutral]",2
1419,MedNLI1419,\nTASK: Please classify the relationship betwe...,entailment,"[entailment, contradiction, neutral]",0
1420,MedNLI1420,\nTASK: Please classify the relationship betwe...,contradiction,"[entailment, contradiction, neutral]",1


In [5]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [6]:
premise_texts_tokens = []
hypothesis_texts_tokens = []
lemmatizer = WordNetLemmatizer()
for query,answer in zip(train_data['query'],train_data['answer']):
    premise,hypothesis = find_pre_and_hyp(query)

    premise = premise.replace('-','')
    hypothesis = hypothesis.replace('-','')
    #premise = re.sub
    hypothesis = 'start ' + hypothesis

    tokens_premise = nltk.word_tokenize(premise)
    tokens_hypothesis = nltk.word_tokenize(hypothesis)

    tokens_premise = [lemmatizer.lemmatize(token.lower()) for token in tokens_premise if token not in string.punctuation]
    tokens_hypothesis = [lemmatizer.lemmatize(token.lower()) for token in tokens_hypothesis if token not in string.punctuation]

    premise_texts_tokens.append(tokens_premise)
    hypothesis_texts_tokens.append(tokens_hypothesis)

In [7]:
def get_vectors_of_given_word(word):
    try:
       return model.wv.get_vector(word)
    except:
       return random.uniform(-1,1,size=(100,))
       #return None

In [8]:
x_premise_arr = []
max_len = 0
for text in premise_texts_tokens:
    x = np.empty((0,100))
    for word in text:
        vectors = get_vectors_of_given_word(word)
        vectors_np = np.array(vectors)
        x = np.vstack([x, vectors_np])
    if max_len < x.shape[0]:
         max_len = x.shape[0]
    x_premise_arr.append(x)

In [9]:
x_hypothesis_arr = []
for text in hypothesis_texts_tokens:
    x = np.empty((0,100))
    for word in text:
        vectors = get_vectors_of_given_word(word)
        vectors_np = np.array(vectors)
        x = np.vstack([x, vectors_np])
    if max_len < x.shape[0]:
         max_len = x.shape[0]
    x_hypothesis_arr.append(x)

In [10]:
y_arr = []
for query,answer in zip(train_data['query'],train_data['answer']):
    if answer == 'entailment':
       y = [1,0,0]
    elif answer == 'neutral':
       y = [0,1,0]
    elif answer == 'contradiction':
       y = [0,0,1]
    else:
        print('should not get here')
    y_arr.append(y)

    #premise,hypothesis = find_pre_and_hyp(query)

In [11]:
for x_premise in x_premise_arr:
    if x_premise.shape[0] == 0:
       print(x_premise.shape)

In [12]:
for x_hypothesis in x_hypothesis_arr:
    if x_hypothesis.shape[0] == 0:
       print(x_hypothesis.shape)

In [13]:
print(len(x_hypothesis_arr))
print(len(x_premise_arr))
print(len(y_arr))

11232
11232
11232


In [14]:
class CustomDataset(Dataset):
    def __init__(self, x_hypothesis_arr , x_premise_arr, y_list):
        self.samples = []
        while x_premise_arr:
            x_pre = x_premise_arr.pop()
            x_hyp = x_hypothesis_arr.pop()
            y = y_list.pop()
            x_pre_tensor = torch.tensor(x_pre,dtype = torch.float32)#.cuda()
            x_hyp_tensor = torch.tensor(x_hyp,dtype = torch.float32)#.cuda()
            y_tensor = torch.tensor(y,dtype = torch.float32)#.cuda()
            self.samples.append((x_pre_tensor,x_hyp_tensor,y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [15]:
train_dataset = CustomDataset(x_hypothesis_arr , x_premise_arr, y_arr)

In [16]:
def get_maxlen(batch,i):
    maxlen = 0
    for tensr in batch:
        if tensr[i].shape[0] > maxlen :
           maxlen = tensr[i].shape[0]
    return maxlen

In [170]:
def collate_fn(batch):
    x_pres = []
    x_hyps = []
    ys = []
    pre_max_len = get_maxlen(batch,0)
    hyp_max_len = get_maxlen(batch,1)
    for tensr in batch:
        x_pre_tensor = tensr[0]
        x_hyp_tensor = tensr[1]
        y_tensor = tensr[2]
        pre_pad_width = pre_max_len - x_pre_tensor.shape[0]
        pre_leftpad_width = pre_pad_width // 2
        pre_rightpad_width = pre_pad_width - pre_leftpad_width
        pre_p1d = (0,0,pre_leftpad_width,pre_rightpad_width)
        paded_x_pre_tensor = F.pad(x_pre_tensor,pre_p1d,"constant",0)
        x_pres.append(paded_x_pre_tensor)
        hyp_pad_width = hyp_max_len - x_hyp_tensor.shape[0]
        hyp_leftpad_width = hyp_pad_width // 2
        hyp_rightpad_width = hyp_pad_width - hyp_leftpad_width
        hyp_p1d = (0,0,hyp_leftpad_width,hyp_rightpad_width)
        paded_x_hyp_tensor = F.pad(x_hyp_tensor,hyp_p1d,"constant",0)
        x_hyps.append(paded_x_hyp_tensor)
        ys.append(y_tensor)
    X_pre = torch.stack(x_pres,dim=0)
    X_hyp = torch.stack(x_hyps,dim=0)
    Y = torch.stack(ys,dim=0)
    return [X_pre,X_hyp,Y]

In [171]:
bsize = 512
train_loader = DataLoader(train_dataset, batch_size=bsize,collate_fn=collate_fn, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=1,collate_fn=collate_fn, shuffle=False)

In [172]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.lstm1 = nn.LSTM(100, 20, 1,batch_first = True)
        self.lstm2 = nn.LSTM(100, 20, 1,batch_first = True)
        self.fc1 = nn.Linear(20, 3)

    def forward(self,x_pre,x_hyp):
        #xpre_h0 = torch.zeros(1, bsize, 20)
        #xpre_c0 = torch.zeros(1, bsize, 20)
        output, (hn, cn) = self.lstm1(x_pre)

        xhyp_h0 = hn
        xhyp_c0 = cn
        output, (hn, cn) = self.lstm1(x_hyp, (xhyp_h0, xhyp_c0))
        a = self.fc1(hn)
        #print(a.shape)
        x = a.reshape([a.shape[1],a.shape[2]])
        #print(x.shape)
        x = nn.functional.softmax(x)
        return x

In [173]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [174]:
model.train()
num_epochs = 50
for epoch in range(num_epochs):
    for x_pre, x_hyp,y in train_loader:
        optimizer.zero_grad()
        outputs = model(x_pre,x_hyp)
        #print(outputs)
        #print(y)
        #print(outputs.shape)
        #print(y.shape)
        loss = criterion(outputs, y)
        #print('==================================================')
        loss.backward()
        optimizer.step()

        #for name, param in model.named_parameters():
            #if param.grad is not None:
                #print(f"Gradient of {name}:")
                #print(param.grad)

        #if (i+1) % 100 == 0:
            #print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')
    print(loss.item())

print('Training complete.')

<ipython-input-172-f48a91d194e6>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


1.0973619222640991
1.0830283164978027
1.0495167970657349
0.9981504082679749
0.967770516872406
0.9494197368621826
0.9471912384033203
0.9629036784172058
0.9347978830337524
0.9527467489242554
0.9331366419792175
0.9038905501365662
0.9047082662582397
0.9223946332931519
0.9166706800460815
0.8896294236183167
0.9228702783584595
0.9342840909957886
0.9054978489875793
0.9053681492805481
0.9105116724967957
0.8892166018486023
0.9000357389450073
0.9123252630233765
0.8893177509307861
0.8792946934700012
0.8913494944572449
0.8755102753639221
0.8810515999794006
0.8788360357284546
0.8704883456230164
0.8633061647415161
0.8842135071754456
0.8776372075080872
0.8761492371559143
0.8317888975143433
0.8817338347434998
0.8675546050071716
0.8901960849761963
0.9025231599807739
0.8466488122940063
0.8622628450393677
0.8611946702003479
0.8774322271347046
0.9048840403556824
0.87325519323349
0.8587046265602112
0.8691462874412537
0.8471463918685913
0.8283083438873291
Training complete.


In [195]:
torch.save(model,'m.pth')